In [4]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d as a3
from matplotlib.path import Path
import matplotlib.patches as patches
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rcParams['text.latex.preamble'] ='\\usepackage{libertine}\n\\usepackage[utf8]{inputenc}'

import seaborn
seaborn.set(style='whitegrid')
seaborn.set_context('notebook')
import random
import PreEmo
import DestiRec

from selection import *
from deap import algorithms, base, benchmarks, tools, creator
import warnings
warnings.filterwarnings("ignore", 'This pattern has match groups')
warnings.filterwarnings("ignore", 'invalid value encountered in true_divide')
warnings.filterwarnings("ignore", 'A class named ')

pd.options.mode.chained_assignment = None
import evaluation

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Assuming given user Input
Duration = 5 weeks
Budget = 4000
Unwanted Regions = [Afghanistan, Pacific]
Preference activities = [hiking, beach, watersports, culinary, entertainment]
Preferred Travel Months = [apr, may, jun]

### Now pre-initialization

First we use the similarity metric to get a ranking of regions
$$
\begin{align*}
sim_{feature} (f_q, f_c) = 1 - \frac{\lvert f_q - f_c \rvert} {max(f_q , f_c)}
\end{align*}
$$

$$
\begin{align*}

similarity (q, c) = \frac{\sum_{i=1}^{n} w_i * sim_i(q_i, c_i)} {\sum_{i=1}^{n} w_i}
\end{align*}
$$

similarity current region and given input
all possible preferences is n
f_q = 1
f_c = actual score for preference i



We do a Max Min Normalization of the scores 

$$
\begin{align*}
X_{ij} = \frac{X_{ij} - X_j^{min}}{X_j^{max}-X_j^{min}}
\end{align*}
$$

Implementing the Moop Problem

$$
\begin{align*}maximize \qquad  &z_k(x) = \sum_{i=1}^n x_i\ s_{ik}, \hspace{1cm} k = 1,...,p \\
    subject \ to \qquad &\sum_{i=1}^n x_i d_i  \leq D \\
    &\sum_{i=1}^n x_i\ b_i \leq B \\
    &\sum_{j \in N} x_{ij} \geq 1 \\ 
    &g_{il}(x_i) \leq \delta,  \hspace{1cm} \forall \ 1 \leq i < l \leq n  \\
    x_i \in \{0,1\}, \qquad &\forall \ 1 \leq i \leq n
\end{align*}
$$

$$ f^\mathrm{penalty}_i(\mathbf{x}) = \Delta_i - w_i d_i(\mathbf{x}) $$

Possible  Categories

['oct' 'beach' 'watersports' 'entertainment' 'wintersports' 'culture'
 'culinary' 'cities & architecture' 'shopping' 'jan' 'feb' 'mar' 'apr'
 'may' 'jun' 'jul' 'aug' 'sep' 'nov' 'dec' 'hiking' 'nature & wildlife'
 'Safety from crime' 'Weeks to Lower Quantile' 'Weeks to Upper Quantile'
 'average weekly cost']

In [ ]:
evaluation.run()